In [5]:
#imports
import nasapy
import getpass
from dotenv import load_dotenv
import os

from langchain_core.tools import tool
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.tools.render import render_text_description_and_args
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_ibm import WatsonxLLM
from datetime import datetime

In [6]:
load_dotenv()

True

In [7]:
credentials = {
    "url": os.environ['WATSONX_URL'],
    "apikey": os.environ['IBM_CLOUD_APIKEY'],
    "project_id": os.environ['WATSONX_PROJECT_ID']
}

In [8]:
n = nasapy.Nasa(key='DEMO_KEY')

In [9]:
param = {
    "decoding_method": "greedy",
    "temperature": 0,
    "min_new_tokens": 5,
    "max_new_tokens": 250,
    "stop_sequences": ["\n\n"]
}

In [10]:
model = WatsonxLLM(
    model_id = "ibm/granite-3-8b-instruct",
    url = credentials.get("url"),
    apikey = credentials.get("apikey"),
    project_id = credentials.get("project_id"),
    params = param
)

In [11]:
model.invoke("What is the date today?")

'\n\nThe date today is [insert current date].'

### Step 5. Define the agent's tools

Unlike traditional LLMs, AI agents can provide more comprehensive responses to diverse tasks through their tool usage, memory, and planning. Agents can use built-in tools such as the Wikipedia API tool available through the langchain_community package. We can also build custom agents that load personalized tools. Our agent in this tutorial will have two custom tools available to use:

get_todays_date() - uses the Python datetime package to return today's date in YYYY-MM-DD format.

get_astronomy_image() - utilizes the NASA API to obtain the Astronomy Picture of the Day. After the tool acquires the image, its URL is returned.

In [12]:
@tool
def get_todays_date() -> str:
    """Get today's date in YYYY-MM-DD format."""
    date = datetime.now().strftime("%Y-%m-%d")
    return date


@tool(return_direct=True)
def get_astronomy_image(date: str):
    """Get NASA's Astronomy Picture of the Day on given date. The date is formatted as YYYY-MM-DD."""
    apod = n.picture_of_the_day(date, hd=True)
    return apod['url']


tools = [get_todays_date, get_astronomy_image]

### Step 6. Establish the prompt template
Next, we will set up a new prompt template to ask multiple questions. This template is more complex. It is refered to as a structured chat prompt and is used for creating agents that have multiple tools available. It will be made up of a system_prompt, a human_prompt and the tools we defined in Step 5.

First, we will set up the system_prompt. This prompt instructs the agent to print its "thought process," which involves the agent's subtasks, the tools that were used and the final output. This gives us insight into the agent's function calling. The prompt also instructs the agent to return its responses in JSON Blob format and to consider information it has stored in its memory.

In [13]:
system_prompt = """Respond to the human as helpfully and accurately as possible. You have access to the following tools: {tools}
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:"
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action.
Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation"""

In [14]:
human_prompt = """{input}
{agent_scratchpad}
(reminder to always respond in a JSON blob)"""

Now, lets finalize our prompt template by adding the tool names, descriptions and arguments using a [partial prompt template](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/partial/?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-ai-agent-python-watsonx). This allows the agent to access the information pertaining to each tool including the intended use cases. This also means we can add and remove tools without altering our entire prompt template.

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human_prompt),
    ]
)

### Step 7. Set up the agent's memory and chain
An important feature of AI agents is their memory. Agents are able to store past conversations and past findings in their memory to improve the accuracy and relevance of their responses going forward. In our case, we will use LangChain's ConversationBufferMemory() as a means of memory storage.

In [16]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

/tmp/ipykernel_12891/3353824168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


And now we can set up a chain with our LangChain agent's scratchpad, memory, prompt and the LLM. We use the AgentExecutor class here to load tools for the agent to use along with the agent itself, an error handling approach, a verbose parameter and memory.

In [17]:
chain = ( RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        chat_history=lambda x: memory.chat_memory.messages,
    )
    | prompt | model | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(agent=chain, tools=tools, verbose=True, memory=memory, handle_parsing_errors=True)

### Step 8. Generate responses with the AI agent

We are now able to ask the agent questions. Recall the agent's previous inability to provide us with the current date. Now that the agent has its tools available to use, let's try asking the same question again.

In [18]:
agent_executor.invoke({"input": "What is today's date?"})



> Entering new AgentExecutor chain...


KeyError: "Input to ChatPromptTemplate is missing variables {'tools', 'tool_names'}.  Expected: ['agent_scratchpad', 'input', 'tool_names', 'tools'] Received: ['input', 'intermediate_steps', 'agent_scratchpad', 'chat_history']\nNote: if you intended {tools} to be part of the string and not a variable, please escape it with double curly braces like: '{{tools}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "